# SynFerm data preparation
#### Targets:
- Import experiment, representation, and target data from db
- Export to CSV

In [2]:
import os
import sys

sys.path.insert(0, os.path.abspath('../..'))
from copy import deepcopy

from IPython.display import SVG, display
import pandas as pd
from rdkit.Chem import (
    Draw,
    rdChemReactions,
    MolFromSmiles
)

from src.util.db_utils import SynFermDatabaseConnection


In [16]:
db = SynFermDatabaseConnection()  # we will use this for various simple queries

In [17]:
res = db.con.execute('SELECT * FROM experiments;').fetchall()
columns = db.con.execute('PRAGMA table_info(experiments);').fetchall()
df_full = pd.DataFrame(res, columns=[c[1] for c in columns])
print(f'Number of reactions (in total): {len(df_full)}')

Number of reactions (in total): 58664


In [19]:
df_valid = df_full.loc[(~df_full["valid"].str.contains("ERROR", na=False)) & (df_full["exp_nr"].between(4, 29))]  # kick out invalid experiments
print(f'Number of valid (non-test) reactions: {len(df_valid)}')

Number of valid (non-test) reactions: 40433


In [23]:
# doublecheck we don't have missing values
df_valid['product_A_lcms_ratio'].isna().sum()

0

Now we have a cleaned dataset.

In [7]:
# first normalize all the lcms columns separately
df = deepcopy(df)
lcms_df = df[['product_A_lcms_ratio', 'product_B_lcms_ratio', 'product_C_lcms_ratio', 'product_D_lcms_ratio', 'product_E_lcms_ratio', 'product_F_lcms_ratio', 'product_G_lcms_ratio', 'product_H_lcms_ratio']]
# df['A_normalized'] = (df['product_A_lcms_ratio']-df['product_A_lcms_ratio'].min())/(df['product_A_lcms_ratio'].max()-df['product_A_lcms_ratio'].min())
df[['A_normalized','B_normalized', 'C_normalized', 'D_normalized', 'E_normalized', 'F_normalized', 'G_normalized', 'H_normalized']] = (lcms_df-lcms_df.min())/(lcms_df.max()-lcms_df.min())
df                                      

,exp_nr,plate_nr,well,lab_journal_number,synthesis_date_unixepoch,initiator,monomer,terminator,initiator_long,monomer_long,...,vl_id,valid,A_normalized,B_normalized,C_normalized,D_normalized,E_normalized,F_normalized,G_normalized,H_normalized
id,,,,,,,,,,,,,,,,,,,,,
9618,5,2,A3,JG253,2021-08-16 22:00:00,I67,M2,T25,Ph023,Fused005,...,200310,heating failure,0.000227,0.000000,0.000335,0.000000,0.013379,0.00000,0.000000,0.000000
9619,5,2,A4,JG253,2021-08-16 22:00:00,I67,M2,T39,Ph023,Fused005,...,200324,heating failure,0.000000,0.000000,0.000000,0.000000,0.001412,0.00000,0.000041,0.000000
9620,5,2,A5,JG253,2021-08-16 22:00:00,I67,M2,T30,Ph023,Fused005,...,200315,heating failure,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
9621,5,2,A6,JG253,2021-08-16 22:00:00,I67,M2,T34,Ph023,Fused005,...,200319,heating failure,0.000000,0.000000,0.000000,0.000095,0.006787,0.00000,0.000000,0.000000
9622,5,2,A7,JG253,2021-08-16 22:00:00,I67,M2,T17,Ph023,Fused005,...,200302,heating failure,0.000429,0.000000,0.000000,0.000252,0.000662,0.00000,0.000000,0.000641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27596,11,6,P4,JG295,2022-01-24 23:00:00,I57,M67,T39,Ph007,Spiro009,...,172649,None,0.000000,0.001072,0.000000,0.023358,0.015007,0.00000,0.000000,0.000000
27597,11,6,P5,JG295,2022-01-24 23:00:00,I57,M67,T30,Ph007,Spiro009,...,172640,None,0.000332,0.000000,0.000000,0.032559,0.000000,0.00000,0.000000,0.000919
27598,11,6,P6,JG295,2022-01-24 23:00:00,I57,M67,T34,Ph007,Spiro009,...,172644,None,0.000896,0.000000,0.000000,0.072766,0.052386,0.00000,0.000181,0.000182


we introduce a "target" column, which is A_normalized binned at 0.01

In [8]:
df["target"] = pd.cut(df["A_normalized"], bins=[-1e-10, 0.01, 1.0000001], labels=[0, 1])
df

,exp_nr,plate_nr,well,lab_journal_number,synthesis_date_unixepoch,initiator,monomer,terminator,initiator_long,monomer_long,...,valid,A_normalized,B_normalized,C_normalized,D_normalized,E_normalized,F_normalized,G_normalized,H_normalized,target
id,,,,,,,,,,,,,,,,,,,,,
9618,5,2,A3,JG253,2021-08-16 22:00:00,I67,M2,T25,Ph023,Fused005,...,heating failure,0.000227,0.000000,0.000335,0.000000,0.013379,0.00000,0.000000,0.000000,0
9619,5,2,A4,JG253,2021-08-16 22:00:00,I67,M2,T39,Ph023,Fused005,...,heating failure,0.000000,0.000000,0.000000,0.000000,0.001412,0.00000,0.000041,0.000000,0
9620,5,2,A5,JG253,2021-08-16 22:00:00,I67,M2,T30,Ph023,Fused005,...,heating failure,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0
9621,5,2,A6,JG253,2021-08-16 22:00:00,I67,M2,T34,Ph023,Fused005,...,heating failure,0.000000,0.000000,0.000000,0.000095,0.006787,0.00000,0.000000,0.000000,0
9622,5,2,A7,JG253,2021-08-16 22:00:00,I67,M2,T17,Ph023,Fused005,...,heating failure,0.000429,0.000000,0.000000,0.000252,0.000662,0.00000,0.000000,0.000641,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27596,11,6,P4,JG295,2022-01-24 23:00:00,I57,M67,T39,Ph007,Spiro009,...,None,0.000000,0.001072,0.000000,0.023358,0.015007,0.00000,0.000000,0.000000,0
27597,11,6,P5,JG295,2022-01-24 23:00:00,I57,M67,T30,Ph007,Spiro009,...,None,0.000332,0.000000,0.000000,0.032559,0.000000,0.00000,0.000000,0.000919,0
27598,11,6,P6,JG295,2022-01-24 23:00:00,I57,M67,T34,Ph007,Spiro009,...,None,0.000896,0.000000,0.000000,0.072766,0.052386,0.00000,0.000181,0.000182,0
